<a href="https://colab.research.google.com/github/bwsi-hadr/2019-student-final-exercise/blob/master/Planning_Response_Phase_Facility_Flooding_and_Set_Cover_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# need to specify location of some certificates for rasterio
!export CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt
!sudo mkdir -p /etc/pki/tls/certs
!sudo cp /etc/ssl/certs/ca-certificates.crt /etc/pki/tls/certs/ca-bundle.crt
try:
  import rasterio
  import rasterio.plot
  import rasterio.merge 
  import rasterio.mask
except:
  !pip install rasterio
  import rasterio
  import rasterio.plot
  import rasterio.merge
  import rasterio.mask
  
try:
  import rasterstats as rs
except:
  !pip install rasterstats  
  import rasterstats as rs
  
try:
  import pyproj
except:
  !pip install pyproj
  import pyproj
  
import networkx as nx
try:
  import osmnx as ox
except:
  # osmnx depends on the system package libspatialindex
  !apt install libspatialindex-dev
  !pip install osmnx
  import osmnx as ox

try: 
  import geopandas as gpd
except: 
  !pip install geopandas 
  import geopandas as gpd
  
try:
  import contextily as ctx 
except:
  # install dependencies for contextily
  !apt install libproj-dev proj-data proj-bin
  !apt install libgeos-dev
  !pip install cython
  !pip install cartopy
  # install contextily
  !pip install contextily==1.0rc1 --no-use-pep517 --no-cache-dir
  import contextily as ctx
  
import fiona
from shapely.geometry import Point, LineString, Polygon
  
import gdal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/BWSI-Remote-Sensing/'Final_exercise'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/BWSI-Remote-Sensing/Final_exercise


In [3]:
!ls game_grid_export

facilities	      flooding.shx	       track.cpg
facilities_stats.csv  game_grid_all_stats.cpg  track.dbf
flooding.cpg	      game_grid_all_stats.dbf  track.prj
flooding.dbf	      game_grid_all_stats.prj  track.qpj
flooding.prj	      game_grid_all_stats.shp  track.shp
flooding.shp	      game_grid_all_stats.shx  track.shx


In [0]:
flooding = gpd.read_file('game_grid_export/flooding.shp')

In [0]:
grid = gpd.read_file('game_grid_export/game_grid_all_stats.shp').to_crs(epsg = 3857)
grid['centroid'] = grid['geometry'].centroid

In [0]:
to_m = lambda km: km * 1000

In [0]:
from shapely.geometry import MultiPolygon
import shapely.ops as shap
#Creates a MultiPolygon using the flood shapely file and the buffer size in meters
def flcreate(fl, i): 
  flooding = fl.copy().to_crs(grid.crs)
  if i == 0:
    flood = flooding['geometry'].unary_union
  else:
    l = [MultiPolygon([flooding.loc[i, 'geometry'], \
                       flooding.loc[i + 1, 'geometry']]).convex_hull \
         for i in range(len(flooding) - 1)]
    flood = shap.cascaded_union(MultiPolygon(l))
    
  return flood

In [0]:
#Outputs a list of facilities in the specified distance from the flood
def uprox(mindist, maxdist, flood):
  l = []
  for i in range(len(fdf_idx)):
    dist = fdf_idx.loc[i, 'geometry'].distance(flood)
    if dist <= maxdist and dist >= mindist: #TODO: mindist <= dist <= maxdist check
      if fdf_idx.loc[i, 'NAME'] not in l:
        l.append(fdf_idx.loc[i, 'NAME'])
  return l

In [0]:
def wcalc_weight(subset, t, cost_ref, threshold):
  s = 0
  for cell in subset:
    if grid.loc[cell, 'Health'] <= threshold:
      s += grid.loc[cell, 'population']
  w = s / cost_ref[t]
  return w

In [0]:
#freq_chart = [0 for _ in range(game_grid.shape[0])]
covered = set()

def wset_cover(budget, facdict, cost_ref, threshold): #facdict = hosp_to_cells, EMS_to_cells, shelt_to_cells, fire_to_cells
  to_ret = set()
  while budget > 0 and len(facdict.items()) != 0:
    max_fac, max_subset = max(facdict.items(), key = lambda tup: \
                               wcalc_weight(tup[1], tup[0][1], cost_ref, threshold))
#     if wcalc_weight(max_subset, max_fac[1], cost_ref, threshold) == 0:
#       max_fac, max_subset = max(facdict.items(), key = lambda tup: \
#                                calc_weight(tup[1], tup[0][1], cost_ref))
    
#Set cover could be wrong because of overlapping areas
#    for cell in max_subset:
#      freq_chart[cell] += 1
#       if freq_chart[cell] == 2:
#         done.add(cell)
    
    budget -= cost_ref[max_fac[1]]
    to_ret.add(max_fac)
    
    for key in facdict:
      facdict[key] -= max_subset
    del facdict[max_fac]
    
  return to_ret

In [0]:
#costs is a list of costs in the order of large hospitals, small hospitals
#large EMS, small EMS, shelters, and fire stations
def wuoptfac (costs, budget, fac_info, threshold = 6):
  large_hos_cost = costs[0]
  small_hos_cost = costs[1]
  large_ems_cost = costs[2]
  small_ems_cost = costs[3]
  shelter_cost = costs[4]
  fstation_cost = costs[5]

  cost_ref = {'smallhosp': small_hos_cost, 'largehosp': large_hos_cost, \
              'smallEMS': small_ems_cost, 'largeEMS': large_ems_cost, \
              'shelter': shelter_cost, 'fstation': fstation_cost}
  fac_to_cells = {}
  for i, row in fac_info.iterrows():
    fac_name = row["NAME"]
    f = ''
    if row['type'] == "small hospital":
      f = 'smallhosp'
    elif row['type'] == "large hospital":
      f = 'largehosp'
    elif row['type'] == "small EMS":
      f = 'smallEMS'
    elif row['type'] == "large EMS":
      f = 'largeEMS'
    elif row['type'] == "shelter":
      f = 'shelter'
    elif row['type'] == "fire station":
      f = 'fstation'
    else:
      return 'no'
    if (fac_name, f) not in fac_to_cells:
      fac_to_cells[(fac_name, f)] = {i}
    else:
      fac_to_cells[(fac_name, f)].add(i)
  
  return wset_cover(budget, fac_to_cells, cost_ref, threshold)
    

In [0]:
def wuabracadabra(costs, budget, mindist, maxdist, fl = flooding, graph_type = 0, threshold = 6):
  l = uprox(mindist, maxdist, flcreate(fl, graph_type))

  x = wuoptfac(costs, budget, fdf_info.loc[fdf_info['NAME'].isin(l)], threshold)
  l = [i[0] for i in x]
  
  return alpha_fdf.loc[l]

In [0]:
#Input the current grid
def main(grid, flooding, costs, budget, mindist, maxdist, graph_type, threshold):
  #Hospitals
  hospital_grid = gpd.read_file('game_grid_export/facilities/hospitals.shp').to_crs(grid.crs)
  hospital_grid.loc[hospital_grid.loc[:, 'BEDS'] >= 200, 'radius'] = 25
  hospital_grid.loc[hospital_grid.loc[:, 'BEDS'] < 200, 'radius'] = 15
  hospital_grid.loc[hospital_grid.loc[:, 'BEDS'] >= 200, 'type'] = "large hospital"
  hospital_grid.loc[hospital_grid.loc[:, 'BEDS'] < 200, 'type'] = "small hospital"
  hospital_grid['health'] = 2

  big_hospitals=hospital_grid[hospital_grid['BEDS']>=200].to_crs(grid.crs)
  small_hospitals=hospital_grid[hospital_grid['BEDS']<200].to_crs(grid.crs)
  hospitals_buffered = hospital_grid.copy().to_crs(grid.crs)
  hospitals_buffered.loc[big_hospitals.index, 'geometry'] = hospitals_buffered.loc[big_hospitals.index,'geometry'].buffer(to_m(25))
  hospitals_buffered.loc[small_hospitals.index, 'geometry'] = hospitals_buffered.loc[small_hospitals.index,'geometry'].buffer(to_m(15))
  hospitals_buffered['centroid'] = hospitals_buffered['geometry'].centroid

  hosp_info = gpd.sjoin(grid, hospitals_buffered, how = 'inner', op = 'intersects')
  hosp_infopop = hosp_info.groupby('NAME').sum()

  alpha_hosp = hospitals_buffered.sort_values('NAME').set_index('NAME')
  alpha_hosp['population'] = hosp_infopop['population'].values
  hosp_idx = alpha_hosp.reset_index()

  #EMS
  EMS_grid = gpd.read_file('game_grid_export/facilities/EMS.shp').to_crs(grid.crs)
  EMS_grid.loc[EMS_grid.loc[:, 'TOTAL_VEHI'] >= 5, 'radius'] = 15
  EMS_grid.loc[EMS_grid.loc[:, 'TOTAL_VEHI'] < 5, 'radius'] = 10
  EMS_grid.loc[EMS_grid.loc[:, 'TOTAL_VEHI'] >= 5, 'type'] = "large EMS"
  EMS_grid.loc[EMS_grid.loc[:, 'TOTAL_VEHI'] < 5, 'type'] = "small EMS"
  EMS_grid['health'] = 1

  big_EMS=EMS_grid[EMS_grid['TOTAL_VEHI']>=5].to_crs(grid.crs)
  small_EMS=EMS_grid[EMS_grid['TOTAL_VEHI']<5].to_crs(grid.crs)
  EMS_buffered = EMS_grid.copy().to_crs(grid.crs)
  EMS_buffered.loc[big_EMS.index, 'geometry'] = EMS_buffered.loc[big_EMS.index,'geometry'].buffer(to_m(15))
  EMS_buffered.loc[small_EMS.index, 'geometry'] = EMS_buffered.loc[small_EMS.index,'geometry'].buffer(to_m(10))
  EMS_buffered['centroid'] = EMS_buffered['geometry'].centroid

  EMS_info = gpd.sjoin(grid, EMS_buffered, how = 'inner', op = 'intersects')
  EMS_infopop = EMS_info.groupby('NAME').sum()

  alpha_EMS = EMS_buffered.sort_values('NAME').set_index('NAME')
  alpha_EMS = alpha_EMS.loc[~alpha_EMS.index.duplicated(keep='first')]
  alpha_EMS.loc[EMS_infopop.index,'population'] = EMS_infopop['population'].values
  alpha_EMS = alpha_EMS.dropna(subset=['population'])
  EMS_idx = alpha_EMS.reset_index()

  #Shelters
  shelt_grid = gpd.read_file('game_grid_export/facilities/shelters.shp').to_crs(grid.crs)
  shelt_grid['radius'] = 3
  shelt_grid['type'] = "shelter"
  shelt_grid['health'] = 1
  shelt_buffered = shelt_grid.copy().to_crs(grid.crs)
  shelt_buffered['geometry'] = shelt_buffered['geometry'].buffer(to_m(3))
  shelt_buffered['centroid'] = shelt_buffered['geometry'].centroid

  shelt_info = gpd.sjoin(grid, shelt_buffered, how = 'inner', op = 'intersects')
  shelt_infopop = shelt_info.groupby('NAME').sum()

  alpha_shelt = shelt_buffered.sort_values('NAME').set_index('NAME')
  alpha_shelt = alpha_shelt.loc[~alpha_shelt.index.duplicated(keep='first')]
  alpha_shelt.loc[shelt_infopop.index,'population'] = shelt_infopop['population'].values
  alpha_shelt = alpha_shelt.dropna(subset=['population'])
  shelt_idx = alpha_shelt.reset_index()

  #Fire Stations
  fire_grid = gpd.read_file('game_grid_export/facilities/fire_stations.shp').to_crs(grid.crs)
  fire_grid['radius'] = 15
  fire_grid['type'] = "fire station"
  fire_grid['health'] = 1
  fire_buffered = fire_grid.copy().to_crs(grid.crs)
  fire_buffered['geometry'] = fire_buffered['geometry'].buffer(to_m(15))
  fire_buffered['centroid'] = fire_buffered['geometry'].centroid

  fire_info = gpd.sjoin(grid, fire_buffered, how = 'inner', op = 'intersects')
  fire_infopop = fire_info.groupby('NAME').sum()

  alpha_fire = fire_buffered.sort_values('NAME').set_index('NAME')
  alpha_fire = alpha_fire.loc[~alpha_fire.index.duplicated(keep='first')]
  alpha_fire.loc[fire_infopop.index,'population'] = fire_infopop['population'].values
  alpha_fire = alpha_fire.dropna(subset=['population'])
  fire_idx = alpha_fire.reset_index()

  #Congregation
  col = ['NAME', 'ID', 'geometry', 'radius', 'population', 'centroid', 'type', 'health']
  hospfac = gpd.GeoDataFrame(columns = col)
  EMSfac = gpd.GeoDataFrame(columns = col)
  sheltfac = gpd.GeoDataFrame(columns = col)
  firefac = gpd.GeoDataFrame(columns = col)


  for i in col:
    hospfac[i] = hosp_idx[i]
    EMSfac[i] = EMS_idx[i]
    sheltfac[i] = shelt_idx[i]
    firefac[i] = fire_idx[i]

  facdf = gpd.GeoDataFrame(columns = col)
  fdf = facdf.append([hospfac, EMSfac, sheltfac, firefac])
  alpha_fdf = fdf.sort_values('NAME').set_index('NAME')
  fdf_idx = alpha_fdf.reset_index()
  
  fdf_info = gpd.sjoin(grid, fdf, how = 'inner', op = 'intersects')
  
  #wuabracadabra
  return wuabracadabra(costs, budget, mindist, maxdist, flooding, graph_type, threshold)

  

In [16]:
main(grid, flooding, [100, 50, 50, 25, 10, 15], 2000, 0, 1000, 0, 6)

/usr/local/lib/python3.6/dist-packages/geopandas/tools/sjoin.py:56: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:3857', 'no_defs': True} != None)
  '(%s != %s)' % (left_df.crs, right_df.crs))
TopologyException: Input geom 0 is invalid: Ring Self-intersection at or near point -7792293.4000208694 5061944.7765096286 at -7792293.4000208694 5061944.7765096286


ValueError: ignored